# Experiment: Check how model influence accuracy

Parameters:
- model size (number of weights / deep)
- embedding size
- training set

In [1]:
# https://huggingface.co/models?pipeline_tag=text-generation&p=1

# MODELS = [
#     'sshleifer/tiny-gpt2', 'lvwerra/gpt2-imdb', 'LorenzoDeMattei/GePpeTto', 'microsoft/CodeGPT-small-py', 'minimaxir/magic-the-gathering',
#     'rinna/japanese-gpt2-medium', 'sberbank-ai/rugpt3small_based_on_gpt2', 'shtoshni/gpt2-chess-uci', 'uer/gpt2-chinese-poem',
#     'ceostroff/harry-potter-gpt2-fanfiction', 'chrisliu298/arxiv_ai_gpt2', 'microsoft/DialoGPT-medium',
#     'gpt2', 'gpt2-medium', 'gpt2-large', 'distilgpt2',
# ]

# TASK_NAMES = ['mnist-add-10', 'mnist-add-100', 'mnist', 'bit-xor-100']

# MODEL_SIZE = {
#     "gpt2": ...,
#     "gpt2-medium": 2_046_043,
#     'tiny-gpt2': 3_911,
# }

In [2]:
# import os
# from tqdm.notebook import tqdm

# jobs = []
# for _ in range(3):
#     for task in TASK_NAMES:
#         for model in MODELS:
#             jobs.append((_, task, model))

# with tqdm(leave=False) as progress:
#     pass
#     for _, task, model in jobs:
#         progress.set_description(f"{task}: {model}")
#         os.system(f"python3 ../run.py --tag exp-model-size --task-name {task} --model {model} ../configs/exp-model-size/gpt2.json")
    

## Collect experiments results

In [11]:
import wandb
import pandas as pd

def get_runs(tag_name: str, xp_name: bool = False):
    api = wandb.Api()

    # Change oreilly-class/cifar to <entity/project-name>
    runs = api.runs("dl-project2/universal-computation-engine")

    results = []
    for run in runs:
        if tag_name in run.tags and 'completed' in run.tags:
            state = run.name, (run.config['xp_name'] if xp_name else run.config['task']+"-"+str(run.config['n'])), run.config['model_name'], run.summary.get('Final Accuracy'), run.config.get('model_weights'), run.config.get('model_all_weights')
            results.append(state)

    df = pd.DataFrame(results, columns=['name', 'task', 'model_name', 'final_accuracy', 'trained_weights', 'all_weights'])
    df['%_weights'] = 100 * df['trained_weights'] / df['all_weights']
    return df

df = get_runs('exp-model-size')
df

,name,task,model_name,final_accuracy,trained_weights,all_weights,%_weights
0,digit-add-100-20210531-0406,digit-add-100,shtoshni/gpt2-chess-uci,2.289050,85349.0,85555301.0,0.099759
1,digit-add-100-20210531-0148,digit-add-100,sberbank-ai/rugpt3small_based_on_gpt2,2.189327,85349.0,125278565.0,0.068127
2,digit-add-100-20210530-2216,digit-add-100,rinna/japanese-gpt2-medium,2.318169,162917.0,336190565.0,0.048460
3,digit-add-100-20210530-2159,digit-add-100,minimaxir/magic-the-gathering,2.211206,12261.0,1691109.0,0.725027
4,digit-add-100-20210530-2055,digit-add-100,microsoft/CodeGPT-small-py,2.802158,85349.0,124290149.0,0.068669
...,...,...,...,...,...,...,...
181,05-27-fpt-20210527-1114,mnist-add-10,gpt2,1.989100,NaN,NaN,NaN
182,05-27-fpt-20210527-1045,mnist-add-10,gpt2,1.999400,NaN,NaN,NaN
183,05-27-fpt-20210527-1038,mnist-add-10,gpt2,2.685200,NaN,NaN,NaN
184,05-27-fpt-20210527-1025,mnist-add-10,gpt2,1.903300,NaN,NaN,NaN


In [7]:
from IPython.display import display

for task in set(df.task):
    df_task = df[df.task == task].groupby(['task', 'model_name']). \
        agg({'final_accuracy': ['size', 'mean', 'std'], 'all_weights': 'mean', 'trained_weights': ['max']}). \
        sort_values(by=('final_accuracy', 'mean'), ascending=('mnist-add' in task))
    display(df_task)

final_accuracy              all_weights  \
                                               size     mean std        mean   
task             model_name                                                    
mnist-add-reg-10 sshleifer/tiny-gpt2              1  7.33633 NaN    104287.0   

                                     trained_weights  
                                                 max  
task             model_name                           
mnist-add-reg-10 sshleifer/tiny-gpt2          3641.0

final_accuracy            \
                                                            size      mean   
task       model_name                                                        
bit-xor-50 sberbank-ai/rugpt3small_based_on_gpt2               2  0.520134   
           lvwerra/gpt2-imdb                                   3  0.510643   
           uer/gpt2-chinese-poem                               2  0.510631   
           microsoft/CodeGPT-small-py                          2  0.509456   
           minimaxir/magic-the-gathering                       2  0.506414   
           gpt2                                                8  0.506349   
           LorenzoDeMattei/GePpeTto                            2  0.505472   
           rinna/japanese-gpt2-medium                          2  0.504588   
           ceostroff/harry-potter-gpt2-fanfiction              1  0.501120   
           distilgpt2                                          1  0.500048   
           chrisliu298/arxiv_ai_gpt2                           1  0.500020   
           shtoshni/gpt2-chess-uci                             2  0.499786   
           gpt2-medium                                         1  0.499766   
           microsoft/DialoGPT-medium                           1  0.499752   
           gpt2-large                                          1  0.499614   
           sshleifer/tiny-gpt2                                 1  0.499372   

                                                              all_weights  \
                                                        std          mean   
task       model_name                                                       
bit-xor-50 sberbank-ai/rugpt3small_based_on_gpt2   0.015107  1.253285e+08   
           lvwerra/gpt2-imdb                       0.016373  1.245302e+08   
           uer/gpt2-chinese-poem                   0.014011  1.032631e+08   
           microsoft/CodeGPT-small-py              0.014105  1.243400e+08   
           minimaxir/magic-the-gathering           0.007020  1.699403e+06   
           gpt2                                    0.009027  1.245367e+08   
           LorenzoDeMattei/GePpeTto                0.006361  1.089793e+08   
           rinna/japanese-gpt2-medium              0.006570  3.362571e+08   
           ceostroff/harry-potter-gpt2-fanfiction       NaN  1.245559e+08   
           distilgpt2                                   NaN  8.202864e+07   
           chrisliu298/arxiv_ai_gpt2                    NaN  7.742260e+08   
           shtoshni/gpt2-chess-uci                 0.000297  8.560520e+07   
           gpt2-medium                                  NaN  3.549779e+08   
           microsoft/DialoGPT-medium                    NaN  3.549779e+08   
           gpt2-large                                   NaN  7.742235e+08   
           sshleifer/tiny-gpt2                          NaN  1.031160e+05   

                                                  trained_weights  
                                                              max  
task       model_name                                              
bit-xor-50 sberbank-ai/rugpt3small_based_on_gpt2         154468.0  
           lvwerra/gpt2-imdb                             154468.0  
           uer/gpt2-chinese-poem                         154468.0  
           microsoft/CodeGPT-small-py                    154468.0  
           minimaxir/magic-the-gathering                  23780.0  
           gpt2                                          154468.0  
           LorenzoDeMattei/GePpeTto                      154468.0  
           rinna/japanese-gpt2-medium                    255076.0  
           ceostroff/harry-potter-gpt2-fanfiction        154468.0  
           distilgpt2                                    136036.0  
           chrisliu298/arxiv_ai_gpt2                     380260.0  
           shtoshni/gpt2-chess-uci                       154468.0  
           gpt2-medium                                   255076.0  
           micr

final_accuracy            \
                                                              size      mean   
task          model_name                                                       
digit-add-100 microsoft/CodeGPT-small-py                         1  2.802158   
              rinna/japanese-gpt2-medium                         1  2.318169   
              shtoshni/gpt2-chess-uci                            1  2.289050   
              minimaxir/magic-the-gathering                      1  2.211206   
              sberbank-ai/rugpt3small_based_on_gpt2              1  2.189327   
              lvwerra/gpt2-imdb                                  1  1.860721   
              LorenzoDeMattei/GePpeTto                           1  1.412846   

                                                         all_weights  \
                                                    std         mean   
task          model_name                                               
digit-add-100 microsoft/CodeGPT-small-py            NaN  124290149.0   
              rinna/japanese-gpt2-medium            NaN  336190565.0   
              shtoshni/gpt2-chess-uci               NaN   85555301.0   
              minimaxir/magic-the-gathering         NaN    1691109.0   
              sberbank-ai/rugpt3small_based_on_gpt2 NaN  125278565.0   
              lvwerra/gpt2-imdb                     NaN  124486757.0   
              LorenzoDeMattei/GePpeTto              NaN  108929381.0   

                                                    trained_weights  
                                                                max  
task          model_name                                             
digit-add-100 microsoft/CodeGPT-small-py                    85349.0  
              rinna/japanese-gpt2-medium                   162917.0  
              shtoshni/gpt2-chess-uci                       85349.0  
              minimaxir/magic-the-gathering                 12261.0  
              sberbank-ai/rugpt3small_based_on_gpt2         85349.0  
              lvwerra/gpt2-imdb                             85349.0  
              LorenzoDeMattei/GePpeTto                      85349.0

final_accuracy            \
                                                          size      mean   
task     model_name                                                        
mnist-10 ceostroff/harry-potter-gpt2-fanfiction              2  0.962850   
         gpt2                                                2  0.959400   
         sberbank-ai/rugpt3small_based_on_gpt2               2  0.953750   
         microsoft/CodeGPT-small-py                          2  0.950650   
         lvwerra/gpt2-imdb                                   2  0.946050   
         gpt2-medium                                         1  0.945800   
         microsoft/DialoGPT-medium                           3  0.943267   
         uer/gpt2-chinese-poem                               2  0.942000   
         rinna/japanese-gpt2-medium                          3  0.932133   
         LorenzoDeMattei/GePpeTto                            2  0.931600   
         distilgpt2                                          4  0.907650   
         shtoshni/gpt2-chess-uci                             2  0.901100   
         minimaxir/magic-the-gathering                       2  0.697950   
         sshleifer/tiny-gpt2                                 2  0.147400   

                                                           all_weights  \
                                                      std         mean   
task     model_name                                                      
mnist-10 ceostroff/harry-potter-gpt2-fanfiction  0.011667  124460554.0   
         gpt2                                    0.004384  124460554.0   
         sberbank-ai/rugpt3small_based_on_gpt2   0.010394  125252362.0   
         microsoft/CodeGPT-small-py              0.034436  124263946.0   
         lvwerra/gpt2-imdb                       0.009405  124460554.0   
         gpt2-medium                                  NaN  354850826.0   
         microsoft/DialoGPT-medium               0.045548  354850826.0   
         uer/gpt2-chinese-poem                   0.038749  103186954.0   
         rinna/japanese-gpt2-medium              0.024082  336155658.0   
         LorenzoDeMattei/GePpeTto                0.013435  108903178.0   
         distilgpt2                              0.009113   81933322.0   
         shtoshni/gpt2-chess-uci                 0.054023   85529098.0   
         minimaxir/magic-the-gathering           0.037123    1686666.0   
         sshleifer/tiny-gpt2                     0.045962     102778.0   

                                                trained_weights  
                                                            max  
task     model_name                                              
mnist-10 ceostroff/harry-potter-gpt2-fanfiction         59146.0  
         gpt2                                           59146.0  
         sberbank-ai/rugpt3small_based_on_gpt2          59146.0  
         microsoft/CodeGPT-small-py                     59146.0  
         lvwerra/gpt2-imdb                              59146.0  
         gpt2-medium                                   128010.0  
         microsoft/DialoGPT-medium                     128010.0  
         uer/gpt2-chinese-poem                          59146.0  
         rinna/japanese-gpt2-medium                    128010.0  
         LorenzoDeMattei/GePpeTto                       59146.0  
         distilgpt2                                     40714.0  
         shtoshni/gpt2-chess-uci                        59146.0  
         minimaxir/magic-the-gathering                   7818.0  
         sshleifer/tiny-gpt2                               84.0

final_accuracy            \
                                                              size      mean   
task         model_name                                                        
mnist-add-10 ceostroff/harry-potter-gpt2-fanfiction              9  1.795033   
             chrisliu298/arxiv_ai_gpt2                           1  1.903600   
             distilgpt2                                          2  1.906850   
             gpt2                                                9  2.045522   
             gpt2-medium                                         5  2.116520   
             sberbank-ai/rugpt3small_based_on_gpt2               3  2.156867   
             lvwerra/gpt2-imdb                                   4  2.177775   
             microsoft/CodeGPT-small-py                          3  2.398633   
             uer/gpt2-chinese-poem                               3  2.420567   
             LorenzoDeMattei/GePpeTto                            4  2.472475   
             microsoft/DialoGPT-medium                           1  2.533500   
             gpt2-large                                          1  2.587800   
             rinna/japanese-gpt2-medium                          2  3.055950   
             minimaxir/magic-the-gathering                       8  3.286125   
             shtoshni/gpt2-chess-uci                             3  3.426467   
             sshleifer/tiny-gpt2                                 7  7.457871   

                                                               all_weights  \
                                                          std         mean   
task         model_name                                                      
mnist-add-10 ceostroff/harry-potter-gpt2-fanfiction  0.142095  125112667.0   
             chrisliu298/arxiv_ai_gpt2                    NaN  775154011.0   
             distilgpt2                              0.048013   82585435.0   
             gpt2                                    0.288062  125112667.0   
             gpt2-medium                             0.506292  355720283.0   
             sberbank-ai/rugpt3small_based_on_gpt2   0.370790  125904475.0   
             lvwerra/gpt2-imdb                       0.239575  125112667.0   
             microsoft/CodeGPT-small-py              0.185609  124916059.0   
             uer/gpt2-chinese-poem                   0.312960  103839067.0   
             LorenzoDeMattei/GePpeTto                0.519733  109555291.0   
             microsoft/DialoGPT-medium                    NaN  355720283.0   
             gpt2-large                                   NaN  775151451.0   
             rinna/japanese-gpt2-medium              0.600829  337025115.0   
             minimaxir/magic-the-gathering           0.350050    1795419.0   
             shtoshni/gpt2-chess-uci                 1.047625   86181211.0   
             sshleifer/tiny-gpt2                     0.124469     104557.0   

                                                    trained_weights  
                                                                max  
task         model_name                                              
mnist-add-10 ceostroff/harry-potter-gpt2-fanfiction       1497691.0  
             chrisliu298/arxiv_ai_gpt2                    1308251.0  
             distilgpt2                                   1479259.0  
             gpt2                                         1497691.0  
             gpt2-medium                                  2046043.0  
             sberbank-ai/rugpt3small_based_on_gpt2        2284123.0  
             lvwerra/gpt2-imdb                            1497691.0  
             microsoft/CodeGPT-small-py                   1497691.0  
             uer/gpt2-chinese-poem                        1497691.0  
             LorenzoDeMattei/GePpeTto                     1497691.0  
             microsoft/DialoGPT-medium                     997467.0  
             gpt2-large                                   1308251.0 

final_accuracy                      \
                                                    size      mean       std   
task        model_name                                                         
bit-xor-100 microsoft/CodeGPT-small-py                 1  0.500538       NaN   
            lvwerra/gpt2-imdb                          2  0.500340  0.000072   
            sshleifer/tiny-gpt2                        2  0.500280  0.000701   
            LorenzoDeMattei/GePpeTto                   2  0.499998  0.001140   
            minimaxir/magic-the-gathering              2  0.499775  0.000352   
            rinna/japanese-gpt2-medium                 1  0.499766       NaN   

                                           all_weights trained_weights  
                                                  mean             max  
task        model_name                                                  
bit-xor-100 microsoft/CodeGPT-small-py     124474568.0        269768.0  
            lvwerra/gpt2-imdb              124671176.0        269768.0  
            sshleifer/tiny-gpt2               103516.0           822.0  
            LorenzoDeMattei/GePpeTto       109113800.0        269768.0  
            minimaxir/magic-the-gathering    1721928.0         43080.0  
            rinna/japanese-gpt2-medium     336436424.0        408776.0

final_accuracy           all_weights trained_weights
                               size mean std         mean             max
task      model_name                                                     
bit-xor-5 gpt2                    1  1.0 NaN  124448261.0         46853.0

final_accuracy                all_weights  \
                                 size      mean std         mean   
task        model_name                                             
digit-add-2 gpt2                    1  0.044575 NaN  124486757.0   

                       trained_weights  
                                   max  
task        model_name                  
digit-add-2 gpt2               85349.0

final_accuracy            \
                                                            size      mean   
task       model_name                                                        
bit-xor-10 gpt2                                                2  0.725190   
           sberbank-ai/rugpt3small_based_on_gpt2               3  0.667093   
           microsoft/CodeGPT-small-py                          3  0.667030   
           uer/gpt2-chinese-poem                               3  0.666637   
           lvwerra/gpt2-imdb                                   5  0.599152   
           minimaxir/magic-the-gathering                       4  0.537960   
           distilgpt2                                          2  0.501680   
           LorenzoDeMattei/GePpeTto                            2  0.501550   
           ceostroff/harry-potter-gpt2-fanfiction              2  0.501540   
           sshleifer/tiny-gpt2                                 3  0.501207   
           rinna/japanese-gpt2-medium                          2  0.500430   
           gpt2-medium                                         1  0.500310   
           shtoshni/gpt2-chess-uci                             3  0.499583   
           gpt2-large                                          1  0.499320   
           microsoft/DialoGPT-medium                           1  0.497900   
           chrisliu298/arxiv_ai_gpt2                           1  0.497600   

                                                              all_weights  \
                                                        std          mean   
task       model_name                                                       
bit-xor-10 gpt2                                    0.317873  1.244636e+08   
           sberbank-ai/rugpt3small_based_on_gpt2   0.288228  1.252529e+08   
           microsoft/CodeGPT-small-py              0.288182  1.242645e+08   
           uer/gpt2-chinese-poem                   0.288459  1.031875e+08   
           lvwerra/gpt2-imdb                       0.169984  1.244606e+08   
           minimaxir/magic-the-gathering           0.074419  1.686866e+06   
           distilgpt2                              0.000071  8.193640e+07   
           LorenzoDeMattei/GePpeTto                0.001216  1.089024e+08   
           ceostroff/harry-potter-gpt2-fanfiction  0.000834  1.244598e+08   
           sshleifer/tiny-gpt2                     0.000763  1.027960e+05   
           rinna/japanese-gpt2-medium              0.003875  3.361546e+08   
           gpt2-medium                                  NaN  3.548549e+08   
           shtoshni/gpt2-chess-uci                 0.000878  8.552962e+07   
           gpt2-large                                   NaN  7.740698e+08   
           microsoft/DialoGPT-medium                    NaN  3.548549e+08   
           chrisliu298/arxiv_ai_gpt2                    NaN  7.740723e+08   

                                                  trained_weights  
                                                              max  
task       model_name                                              
bit-xor-10 gpt2                                           62228.0  
           sberbank-ai/rugpt3small_based_on_gpt2          62228.0  
           microsoft/CodeGPT-small-py                     62228.0  
           uer/gpt2-chinese-poem                          62228.0  
           lvwerra/gpt2-imdb                              62228.0  
           minimaxir/magic-the-gathering                   8340.0  
           distilgpt2                                     43796.0  
           LorenzoDeMattei/GePpeTto                       62228.0  
           ceostroff/harry-potter-gpt2-fanfiction         62228.0  
           sshleifer/tiny-gpt2                              102.0  
           rinna/japanese-gpt2-medium                    132116.0  
           gpt2-medium                                   132116.0  
           shtoshni/gpt2-chess-uci                        62228.0  
           gpt2

# Task-2

In [43]:
df = get_runs('task-2', True)
df

,name,task,model_name,final_accuracy,trained_weights,all_weights,%_weights
0,cyp3a4-unfreeze-20210610-1207,cyp3a4-unfreeze,gpt2,0.600060,125209346,125209346,100.0
1,cyp3a4-unfreeze-20210610-1202,cyp3a4-unfreeze,gpt2,0.600855,125209346,125209346,100.0
2,cyp3a4-unfreeze-20210610-1153,cyp3a4-unfreeze,gpt2,0.653816,125209346,125209346,100.0
3,mnist-add-unfreeze-20210610-1029,mnist-add-unfreeze,gpt2,6.081800,125112667,125112667,100.0
4,mnist-add-unfreeze-20210610-1025,mnist-add-unfreeze,gpt2,8.185100,125112667,125112667,100.0
5,speech-commands-unfreeze-20210610-1025,speech-commands-unfreeze,gpt2,0.034100,130611491,130611491,100.0
6,speech-commands-unfreeze-20210610-1020,speech-commands-unfreeze,gpt2,0.039700,130611491,130611491,100.0
7,speech-commands-unfreeze-20210610-1016,speech-commands-unfreeze,gpt2,0.035700,130611491,130611491,100.0
8,mnist-add-reg-unfreeze-20210610-1012,mnist-add-reg-unfreeze,gpt2,7.430690,125043457,125043457,100.0
9,cifar10-unfreeze-20210610-1007,cifar10-unfreeze,gpt2,0.200700,124485130,124485130,100.0


In [44]:
df_task = df.groupby(['task', 'model_name']). \
    agg({'final_accuracy': ['size', 'mean', 'std'], 'all_weights': 'max', 'trained_weights': ['max']}). \
    sort_values(by=('final_accuracy', 'mean'), ascending=('mnist-add' in task))
display(df_task)

final_accuracy                      \
                                              size      mean       std   
task                     model_name                                      
mnist-add-reg-unfreeze   gpt2                    3  7.412223  0.031683   
mnist-add-unfreeze       gpt2                    5  6.557100  0.949748   
cyp3a4-unfreeze          gpt2                    3  0.618243  0.030809   
mnist-unfreeze           gpt2                    6  0.607600  0.115499   
cifar10-unfreeze         gpt2                    3  0.217300  0.015617   
speech-commands-unfreeze gpt2                    5  0.038040  0.003815   

                                    all_weights trained_weights  
                                            max             max  
task                     model_name                              
mnist-add-reg-unfreeze   gpt2         125043457       125043457  
mnist-add-unfreeze       gpt2         125112667       125112667  
cyp3a4-unfreeze          gpt2         125209346       125209346  
mnist-unfreeze           gpt2         124460554       124460554  
cifar10-unfreeze         gpt2         124485130       124485130  
speech-commands-unfreeze gpt2         130611491       130611491

In [42]:
df_task.dtypes

final_accuracy   size      int64
                 mean    float64
                 std     float64
all_weights      max       int64
trained_weights  max       int64
dtype: object

In [39]:
print(df_task.to_markdown())

|                                      |   ('final_accuracy', 'size') |   ('final_accuracy', 'mean') |   ('final_accuracy', 'std') |   ('all_weights', 'max') |   ('trained_weights', 'max') |
|:-------------------------------------|-----------------------------:|-----------------------------:|----------------------------:|-------------------------:|-----------------------------:|
| ('mnist-add-reg-unfreeze', 'gpt2')   |                            3 |                     7.41222  |                  0.0316831  |              1.25043e+08 |                  1.25043e+08 |
| ('mnist-add-unfreeze', 'gpt2')       |                            5 |                     6.5571   |                  0.949748   |              1.25113e+08 |                  1.25113e+08 |
| ('cyp3a4-unfreeze', 'gpt2')          |                            3 |                     0.618243 |                  0.0308091  |              1.25209e+08 |                  1.25209e+08 |
| ('mnist-unfreeze', 'gpt2')           |     

# Task 4

In [45]:
df = get_runs('task-4', True)
df

,name,task,model_name,final_accuracy,trained_weights,all_weights,%_weights
0,cyp3a4-vit-20210610-1302,cyp3a4-vit,ViT-B_16,0.715322,807938,86613506,0.932808
1,cyp3a4-vit-20210610-1259,cyp3a4-vit,ViT-B_16,0.748510,807938,86613506,0.932808
2,cyp3a4-vit-20210610-1255,cyp3a4-vit,ViT-B_16,0.752484,807938,86613506,0.932808
3,speech-commands-vit-20210610-1227,speech-commands-vit,ViT-B_16,0.361600,127523,85933091,0.148398
4,mnist-add-reg-vit-20210610-1146,mnist-add-reg-vit,ViT-B_16,7.296249,642049,86447617,0.742703
5,speech-commands-vit-20210610-1141,speech-commands-vit,gpt2,0.076800,6210083,130611491,4.754622
6,speech-commands-vit-20210610-1136,speech-commands-vit,gpt2,0.069200,6210083,130611491,4.754622
7,mnist-add-vit-20210610-1130,mnist-add-vit,ViT-B_16,2.023900,711259,86516827,0.822105
8,mnist-add-vit-20210610-1118,mnist-add-vit,ViT-B_16,1.558800,711259,86516827,0.822105
9,cifar10-vit-20210610-1057,cifar10-vit,ViT-B_16,0.445800,83722,85889290,0.097477


In [46]:
df_task = df.groupby(['task', 'model_name']). \
    agg({'final_accuracy': ['size', 'mean', 'std'], 'all_weights': 'max', 'trained_weights': ['max']}). \
    sort_values(by=('final_accuracy', 'mean'), ascending=('mnist-add' in task))
display(df_task)

final_accuracy                     all_weights  \
                                         size      mean       std         max   
task                model_name                                                  
mnist-add-reg-vit   ViT-B_16                1  7.296249       NaN    86447617   
mnist-add-vit       ViT-B_16                2  1.791350  0.328875    86516827   
cyp3a4-vit          ViT-B_16                3  0.738772  0.020405    86613506   
mnist-vit           ViT-B_16                3  0.735967  0.062662    85864714   
cifar10-vit         ViT-B_16                1  0.445800       NaN    85889290   
speech-commands-vit ViT-B_16                1  0.361600       NaN    85933091   
                    gpt2                    2  0.073000  0.005374   130611491   

                               trained_weights  
                                           max  
task                model_name                  
mnist-add-reg-vit   ViT-B_16            642049  
mnist-add-vit       ViT-B_16            711259  
cyp3a4-vit          ViT-B_16            807938  
mnist-vit           ViT-B_16             59146  
cifar10-vit         ViT-B_16             83722  
speech-commands-vit ViT-B_16            127523  
                    gpt2               6210083